<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/neox_python3.8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It is not obvious how to downgrade the Colab Python from 3.10 to 3.8 for GPT-NeoX There are some ideas at https://stackoverflow.com/questions/60775160/install-python-3-8-kernel-in-google-colaboratory It is not possible according to https://github.com/googlecolab/colabtools/issues/4212 So we will try with a venv

In [1]:
import sys
print(sys.version)

3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]


In [2]:
# Update and install Python 3.8
!apt-get update -y
!apt-get install python3.8 python3.8-venv python3.8-dev -y


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,071 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
G

In [8]:
%%bash
python3.8 -m venv myenv
curl -O https://bootstrap.pypa.io/get-pip.py
python get-pip.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2222k  100 2222k    0     0  5036k      0 --:--:-- --:--:-- --:--:-- 5039k


In [12]:
%%bash
source /content/myenv/bin/activate
python --version
pip --version


Python 3.8.20
pip 23.0.1 from /content/myenv/lib/python3.8/site-packages/pip (python 3.8)


In [13]:
!git clone https://github.com/EleutherAI/gpt-neox.git
%cd /content/gpt-neox

Cloning into 'gpt-neox'...
remote: Enumerating objects: 19481, done.
remote: Counting objects: 100% (1684/1684), done.
remote: Compressing objects: 100% (812/812), done.
remote: Total 19481 (delta 1241), reused 1196 (delta 864), pack-reused 17797 (from 1)
Receiving objects: 100% (19481/19481), 113.67 MiB | 30.70 MiB/s, done.
Resolving deltas: 100% (14098/14098), done.
/content/gpt-neox


In [ ]:
!ls

CITATION.cff		      docker-compose.yml  images	 prepare_data.py  tools
configs			      Dockerfile	  LICENSE	 README.md	  train.py
CONTRIBUTING.md		      eval.py		  MANIFEST.in	 README-MUP.md
deepy.py		      eval_tasks	  megatron	 requirements
docker-compose-dockerhub.yml  generate.py	  post-training  tests


In [ ]:
%%bash
source /content/myenv/bin/activate
pip install -r requirements/requirements.txt

We have Python 3.8 in the virtual environment but we can install matching versions of packages in the 3.10 environment to do some scripting in notebook cells (e.g. if they are not running GPT-NeoX)

In [ ]:
%pip install -r requirements/requirements.txt

In [ ]:
import requests
from pathlib import Path

# Download Shakespeare data
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
Path("shakespeare.txt").write_text(response.text)

# Split text into chunks
with open("shakespeare.txt") as f:
    text = f.read()

chunk_size = 256
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Save as JSONL format
with open("shakespeare.jsonl", "w") as f:
    for chunk in chunks:
        f.write(f'{{"text": "{chunk}"}}\n')

In [ ]:
from tokenizers import ByteLevelBPETokenizer

import os

# Create the directory if it doesn't exist
os.makedirs("tokenizer", exist_ok=True)

tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=["shakespeare.txt"], vocab_size=5000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
tokenizer.save_model("tokenizer")

['tokenizer/vocab.json', 'tokenizer/merges.txt']

In [ ]:
import yaml

# Define the configuration data as a dictionary
config_data = {
    "vocab-file": "shakespeare_tokenizer.json",
    "hidden-size": 384,
    "num-attention-heads": 6,
    "num-layers": 6,
    "max-position-embeddings": 256,
    "micro-batch-size": 32,
    "seq-length": 256,
    "train-iters": 5000,
    "lr-decay-iters": 5000,
    "lr": 6e-4,
    "min-lr": 6e-5,
    "warmup": 0.01,
    "adam-beta1": 0.9,
    "adam-beta2": 0.95,
    "weight-decay": 0.1,
    "clip-grad": 1.0,
    "fp16": True
}

# Write the dictionary to a YAML file
with open("shakespeare_config.yml", "w") as f:
    yaml.dump(config_data, f, default_flow_style=False)


In [ ]:
%run deepy.py train.py -d configs shakespeare_config.yml